# Runner for Experiment 0 Part 1

In [1]:
import pandas as pd

from qiskit import assemble
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import CspRbsLayout
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.test.mock.backends import FakeManhattan


from exp.custom_passmanager import custom_pass_manager
from exp.circuit_generators import graphstate_manhatten_ring
from exp.write_to_df import write_to_dataframe

In [2]:
exp_path = "./data/"
num_samples = 1
num_qubits = 12

In [3]:
backend = FakeManhattan()
coupling_map = CouplingMap(backend.configuration().coupling_map)
backend_properties = backend.properties()

In [12]:
def exp0_1_evaluate(circuit, solution_limit, backend, ideal=True, shots=16384, seed=None, layout_reps=100):

    layout_pass = CspRbsLayout(coupling_map, seed=seed, call_limit=None, time_limit=None, \
                               solution_limit=solution_limit, iteration_limit=1, backend_prop=backend.properties())

    if ideal:
        simulator = QasmSimulator(configuration=backend.configuration(),
                                  method='stabilizer')
    else:
        noise_model = NoiseModel.from_backend(backend, thermal_relaxation=False)
        simulator = QasmSimulator(configuration=backend.configuration(),
                                  noise_model=noise_model,
                                  method='stabilizer')

    passmanager = custom_pass_manager(backend, layout_pass, layout_reps=layout_reps, seed=seed)

    times = {}
    count_ops_after_map = {}

    def callback(**kwargs):
        times[kwargs['pass_'].name()] = times.get(kwargs['pass_'].name(), 0) + kwargs['time']
        if kwargs['pass_'].name() == 'LookaheadSwap':
            count_ops_after_map.update(kwargs['dag'].count_ops())

    transpiled = passmanager.run(circuit, callback=callback)

    needed_swaps = 0 if passmanager.property_set['is_swap_mapped'] else count_ops_after_map['swap']

    qobj = assemble(transpiled, backend, shots=shots, seed_simulator=seed)
    result = simulator.run(qobj).result()
    counts = result.get_counts()
    time = times[layout_pass.name()] / layout_reps

    return counts, time, needed_swaps

In [13]:
def exp0_1_run(solution_limit, exp_path, num_samples=100, num_qubits=12, num_shots=16384):
    backend = FakeManhattan()
    circuit = graphstate_manhatten_ring_hm(num_qubits)

    df = pd.DataFrame(columns=["seed", "state", "ideal_count", "noise_count", \
                               "ideal_time", "noise_time", "ideal_swaps_needed", "noise_swaps_needed"])
    
    for num_index in range(num_samples):
        seed = num_index
        ideal_counts, ideal_time, swaps_needed_ideal = exp0_1_evaluate(
            circuit, solution_limit, backend=backend, ideal=True, shots=num_shots, seed=seed)
        noise_counts, noise_time, swaps_needed_noise = exp0_1_evaluate(
            circuit, solution_limit, backend=backend, ideal=False, shots=num_shots, seed=seed)
        total_result = {'seed': seed,
                        'ideal_counts': ideal_counts,
                        'noise_counts': noise_counts,
                        'ideal_time': ideal_time,
                        'noise_time': noise_time,
                        'ideal_swaps_needed': swaps_needed_ideal,
                        'noise_swaps_needed': swaps_needed_noise}

        df_loc = write_to_dataframe(total_result)
        df = df.append(df_loc)

    df.to_csv(exp_path + f'exp0_1_s{solution_limit}.csv')

In [15]:
solution_limit_max = 21
num_samples = 100
for solution_limit in range(1, solution_limit_max):
    exp0_1_run(solution_limit, exp_path, num_samples=num_samples)